In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

In [ ]:
device = 'cuda' # 'cpu'

In [ ]:
# Prepare samples
samples_inp = torch.tensor([[1,2,3],[1,1,1],[4,4,4],[9,9,9],[3,2,1],[0,0,1],[0,0,0]],dtype=torch.float).to(device)
samples_out = torch.tensor([[123],[111],[444],[999],[321],[1],[0]],dtype=torch.float).to(device)
#samples_inp = torch.tensor([[1,2,3],[1,1,1],[4,4,4],[9,9,9]],dtype=torch.float).to(device)
#samples_out = torch.tensor([[123],[111],[444],[999]],dtype=torch.float).to(device)

In [ ]:
class SingleNeuron(nn.Module):
    # initialization
    def __init__(self, num_inputs):
        super().__init__()
        self.neuron = nn.Linear(num_inputs, 1)
    # forward
    def forward(self, x):
        return self.neuron(x)

In [ ]:
# Define model
model = SingleNeuron(3).to(device)
print(list(model.parameters()))

In [ ]:
# Define loss function
loss_function = nn.MSELoss(reduction='sum')

In [ ]:
# Define optimizer
optimizer = optim.Adam(model.parameters())

In [ ]:
# Training
num_epochs = 130000
history = []
t0 = time.time()
for t in range(num_epochs):

    # Forward pass
    out = model(samples_inp)
    loss = loss_function(out, samples_out)
    if t % 100 == 0:
        print(t, loss.cpu().item(), list(next(model.parameters()).clone().detach().cpu().numpy()))
    history.append(loss.cpu().item())

    # Reset gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Update model parameters (weights)
    optimizer.step()

t1 = time.time()
print(f"elapsed: {t1-t0:.4f}s")

In [ ]:
# Saving
torch.save(model.state_dict(), 'single-neuron.pth') # weights only
torch.save(model,'single-neuron.pt') # whole model

In [ ]:
# Print weighs
for param in model.parameters():
    print(param.data)

In [ ]:
# test
result = model(torch.tensor([[7,7,7]],dtype=torch.float).to(device))
print('test result',result[0].item())

In [ ]:
# optional only
import matplotlib.pyplot as plt
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history,'r',linewidth=3.0)
plt.legend(['Training loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curve',fontsize=16)
#
plt.savefig('fig.png')